In [1]:
# All libraries:

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd
import datetime as dt
import requests
import mysql.connector

import webbrowser


Probeabfrage: https://www.chrono24.at/search/index.htm?braceletMaterial=410&caseMaterials=9&currencyId=EUR&dosearch=true&manufacturerIds=221&maxAgeInDays=0&models=3&pageSize=60&redirectToSearchIndex=true&resultview=block&sortorder=0

In [3]:
# Connect to database:


conn = mysql.connector.connect(
    host= "127.0.0.1",
    user = "watchsearch", 
    password = "watchsearch",
    database = "watchsearch"
)

cur = conn.cursor()

cur.execute('''SHOW TABLES''')
for x in cur:
    print(x)

cur.execute('''SELECT * FROM watch''')
result = cur.fetchall()


conn.commit

print(result)




('user',)
('watch',)
[(1, '3', '221', None, None, None, None, None, '410', '09', None, None, None, None)]


Note: you may need to restart the kernel to use updated packages.


In [ ]:
#insert to database


cur.execute('''INSERT INTO table VALUES(?, ?, ?)''', variable1, variable2, variable3)

## nachdem alles erledigt ist
conn.commit
          
          
          
## und als letztes unbedingt:
          conn.close



In [4]:
# Build URL:



cur.execute('''SELECT name FROM watch WHERE id=1''')
c = cur.fetchone()
name = str(c[0])

cur.execute('''SELECT brand FROM watch WHERE id=1''')
c = cur.fetchone()
brand = str(c[0])

cur.execute('''SELECT material FROM watch WHERE id=1''')
c = cur.fetchone()
bracelet = str(c[0])

cur.execute('''SELECT color FROM watch WHERE id=1''')
c = cur.fetchone()
caseMaterial = str(c[0])

URL = 'https://www.chrono24.at/search/index.htm?dosearch=true&' + 'braceletMaterial=' + bracelet + '&caseMaterials=' + caseMaterial + '&manufacturerIds=' + brand + '&maxAgeInDays=0' + '&models=' + name +'&pageSize=60&redirectToSearchIndex=true&resultview=block&sortorder=0'

print(URL)

https://www.chrono24.at/search/index.htm?dosearch=true&braceletMaterial=410&caseMaterials=09&manufacturerIds=221&maxAgeInDays=0&models=3&pageSize=60&redirectToSearchIndex=true&resultview=block&sortorder=0


3 = GMT-Master
221 = Rolex
09 = Gold/Stahl
410 = Gold/Stahl

In [6]:
# Scrape URL:


html = requests.get(URL)


bs = BeautifulSoup(html.content, 'html.parser')

xl = []

for entry in bs.find_all('a', class_ = 'article-item block-item rcard'):
    link = entry['href']
    xl.append(link)
    #print(link)
    
# jetzt muss man die links in einer Liste speichern    
xl    


['/rolex/rolex-gmt-master-1675-nipple-dial-stahl--gold-jubilee-band-bestzustand-unpoliert--2-nos-inlay--id26618914.htm',
 '/rolex/gmt-master-tiger-eye--nipple-dial-stahl--gold-oyster-perpetual-16753--id26499358.htm',
 '/rolex/gmt-master-ii-tigerauge---ref-16753---edelstahl--18k-gelbgold---jubileeband---automatik---plexiglas---jahr-ca-19811982--id23551139.htm',
 '/rolex/rolex-gmt-master--id26324915.htm',
 '/rolex/oyster-perpetual-gmt-master-vintage-1984-ref-16753--id26310571.htm',
 '/rolex/rolex-gmt-master-full-set-unpolished-unpoliert-lc100--id26440454.htm',
 '/rolex/rolex-gmt-master--id23891881.htm',
 '/rolex/gmt-master---stahl--gelbgold---armband-stahl--gelbgold--jubile---40mm---sehr-gut---vintage--id23586869.htm',
 '/rolex/rolex-gmt-master-steel-and-gold-ref1675-brown-nipple-dial-from-1975--id26508278.htm',
 '/rolex/gmt-master-herren-stahlgold---ref16753---top-zustand--id26019939.htm',
 '/rolex/gmt-master-tiger-eye-nipple-dial--id26064074.htm',
 '/rolex/rolex-gmt-master--id21810003.

In [ ]:
# URL wieder zusammenbauen:

directlink = "https://www.chrono24.at" + entry

In [8]:
# Create Table
sql = '''CREATE TABLE ROLEXMASTER (
    LINK TEXT UNIQUE
    )'''
cur.execute(sql)    


## Alternative for: Create Table
sql = '''CREATE TABLE ROLEXMASTER (
    LINK TEXT UNIQUE,
    date DATE
    )'''
cur.execute(sql)   

In [20]:

for entry in bs.find_all('a', class_ = 'article-item block-item rcard'):
    link = entry['href']
    cur.execute("""INSERT INTO ROLEXMASTER (LINK) VALUES (%s)""", (link))


conn.commit()


ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near '%s)' at line 1